# ComCam Science Programs Survey Progress

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
import os
os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

## All ComCam Exposures

In [ ]:
instrument = 'lsstcomcam'

visits_query = f'''
    SELECT * FROM cdb_{instrument}.exposure
'''

visits = client.query(visits_query).to_pandas()

In [ ]:
for img_type in np.unique(visits['img_type']):
    print(img_type, np.sum(visits['img_type'] == img_type))

## Science Program Exposures

In [ ]:
instrument = 'lsstcomcam'

visits_query = f'''
    SELECT * FROM cdb_{instrument}.exposure
    WHERE img_type IN ('OBJECT', 'ACQ')
    AND science_program IN ('BLOCK-320', 'PP-SURVEY', 'BLOCK-T248')
'''

visits = client.query(visits_query).to_pandas()

In [ ]:
visits.columns

In [ ]:
visits

In [ ]:
np.unique(visits['img_type'])

In [ ]:
np.unique(visits['science_program'])

### Distribution by Band

In [ ]:
bands = {'u': 'purple', 'g': 'blue', 'r': 'green', 'i': 'cyan', 'z': 'orange', 'y': 'red'}

In [ ]:
for band in bands:
    print(band, np.sum(visits['band'] == band))

### Distribution by day_obs

In [ ]:
for day_obs in np.unique(visits['day_obs']):
    print(day_obs, np.sum(visits['day_obs'] == day_obs))

In [ ]:
visits.value_counts('day_obs').values

In [ ]:
from astropy.time import Time

In [ ]:
def dayObsToMJD(day_obs):
    year = np.floor(day_obs / 10000).astype(int)
    month = np.floor((day_obs % 10000) / 100).astype(int)
    day = day_obs % 100
    time = Time({'year': year, 'month': month, 'day': day}, format='ymdhms')
    return time.mjd

In [ ]:
def MJDToDayObs(mjd):
    time = Time(mjd, format='mjd')
    return np.array([_.iso.split()[0].replace('-', '') for _ in time])

In [ ]:
#year = np.floor(visits['day_obs'].values / 10000).astype(int)

In [ ]:
#month = np.floor((visits['day_obs'].values % 10000) / 100).astype(int)

In [ ]:
#day = visits['day_obs'].values % 100

In [ ]:
#Time({'year': year, 'month': month, 'day': day}).mjd

In [ ]:
#mjd = np.unique(dayObsToMJD(visits['day_obs']))

In [ ]:
#MJDToDayObs(mjd)

In [ ]:
#np.unique(visits['day_obs'])

In [ ]:
#dayObstoMJD(visits.value_counts('day_obs').index.values)

In [ ]:
#np.arange(np.min(visits['day_obs']), np.max(visits['day_obs']) + 2)[::2]

In [ ]:
mjd = dayObsToMJD(visits['day_obs'])
xticks = np.arange(np.min(mjd), np.max(mjd) + 2)[::2]
xtick_labels = MJDToDayObs(xticks)

plt.figure()
plt.scatter(dayObstoMJD(visits.value_counts('day_obs').index.values), 
            visits.value_counts('day_obs').values, c='black', label='Total')
for band in bands:
    selection = (visits['band'] == band)
    plt.scatter(dayObstoMJD(visits[selection].value_counts('day_obs').index.values), 
                visits[selection].value_counts('day_obs').values, 
                c=bands[band], label=band)

plt.legend(loc='upper left')
plt.xticks(xticks, xtick_labels, rotation=45.)
plt.ylabel('Visit Count')
plt.grid()
plt.tight_layout()

### Distribution by target

In [ ]:
from astropy.coordinates import SkyCoord
from astroplan import FixedTarget

targets = [
    FixedTarget(SkyCoord(6.022329,-72.081444, unit='deg', frame='icrs'), name='47_Tuc'),
    FixedTarget(SkyCoord(37.9, 7.0, unit='deg', frame='icrs'), name='Rubin_SV_38_7'),
    FixedTarget(SkyCoord(39.9971, -34.4492, unit='deg', frame='icrs'), name='Fornax_dSph'),
    FixedTarget(SkyCoord(53.125, -28.1, unit='deg', frame='icrs'), name='ECDFS'),
    FixedTarget(SkyCoord(59.1004, -48.73, unit='deg', frame='icrs'), name='EDFS_ComCam'),
    FixedTarget(SkyCoord(95.0, -25.0, unit='deg', frame='icrs'), name='Rubin_SV_095_-25'),
    FixedTarget(SkyCoord(106.23, -10.51, unit='deg', frame='icrs'), name='Seagull'),
]

In [ ]:
visits_coord = SkyCoord(visits['s_ra'], visits['s_dec'], unit='deg', frame='icrs')

In [ ]:
for target in targets:
    print(target.name)
    angsep = target.coord.separation(visits_coord).deg
    selection_angle = angsep < 2.

    for band in bands:
        selection_band = (visits['band'] == band)
        print(band, np.sum(selection_angle & selection_band))

    print('')

In [ ]:
selection_day_obs = (visits['day_obs'] == 20241203)

for target in targets:
    print(target.name)
    angsep = target.coord.separation(visits_coord).deg
    selection_angle = angsep < 2.

    for band in bands:
        selection_band = (visits['band'] == band)
        print(band, np.sum(selection_angle & selection_band & selection_day_obs))

    print('')

### Distribution by target, band, and airmass

In [ ]:
import astropy.units as u

In [ ]:
(1. * u.day).to(u.second).value

In [ ]:
for target in targets:

    angsep = target.coord.separation(visits_coord).deg
    selection_angle = angsep < 2.

    if not np.any(selection_angle):
        continue
    
    plt.figure()
    for band in bands:
        selection_band = (visits['band'] == band) 
        if np.any(selection_angle & selection_band):
            #if target.name == 'Rubin_SV_095_-25':
            #    #print(90. - visits['altitude'][selection_angle & selection_band])
            #    for ii in range(0, len(visits[selection_angle & selection_band])):
            #        print(
            #            90. - visits['altitude'][selection_angle & selection_band].values[ii], 
            #            visits['azimuth'][selection_angle & selection_band].values[ii],
            #            (1. * u.day).to(u.second).value * (visits['obs_start_mjd'][selection_angle & selection_band].values[ii + 1] - visits['obs_start_mjd'][selection_angle & selection_band].values[ii])
            #        )
            #print(band, np.sum(selection & selection_band))
            plt.scatter(dayObstoMJD(visits['day_obs'][selection_angle & selection_band]), 
                        # visits['exp_midpt_mjd'][selection_angle & selection_band],
                        visits['airmass'][selection_angle & selection_band], 
                        c=bands[band], marker='_', label=band)
    plt.legend(loc='lower left')
    #plt.xlabel('MJD')
    plt.xlim(np.min(xticks) - 0.5, np.max(xticks) + 0.5)
    plt.xticks(xticks, xtick_labels, rotation=45.)
    plt.ylabel('Airmass')
    #xtick_labels = [_.split()[0] for _ in Time(plt.xticks()[0], format='mjd').iso]
    #plt.xticks(plt.xticks()[0], xtick_labels, rotation=45)
    plt.ylim(1.5, 1.0)
    plt.grid()
    plt.title(target.name)
    plt.tight_layout()